<img align="right" src="images/tf.png" width="200"/>
<img align="right" src="images/huc.png" width="200"/>
<img align="right" src="images/logo.png" width="200"/>

---

To get started: consult [start](start.ipynb)

---

# Named Entities

A research group has applied a NER-algorithm to this corpus (Named Entity Recognition) and 
delivered the results as Text-Fabric features in 
[cltl/voc-missives](https://github.com/cltl/voc-missives).

We can use these shared features, they are in `export/tf` and we see that they have been produced
against version `0.8.1` of the corpus data.

This is an example of the practice of using research results that others have shared.

Note that we do not have to do a manual effort to get the data and to integrate it in the corpus.
We refer to them by their location on GitHub, and Text-Fabric does the rest.

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import collections

from tf.app import use

Note that we draw in an earlier version, in which there were still some encoding problems.
In later versions these problems are gone.
In the [porting](porting.ipynb) notebook we'll show how we can carry over
the entity features from an older version to a newer version.

In [5]:
A = use(
    "clariah/wp6-missieven",
    # version="0.8.1",
    version="1.0",
    # mod="/voc-missives/export/tf", # using my fork until a pull-request has been accepted
    # mod="cltl/voc-missives/export/tf",
    mod="cltl/voc-missives/data/tf_export/export:clone",
    hoist=globals(),
)

Above you see a new section in the feature list that you can expand to see
which features that module contributed.

Now, suppose did not know much about these feature, then we would like to do a few basic checks.

A good start it to do inspect a frequency list of the values of the new features,
and then to perform a query looking for the nodes that have these features.

## Entities

First the identities:

In [6]:
F.entityId.freqList()[0:20]

(('e_t_1_3_1', 7),
 ('e_t_3_11_2', 7),
 ('e_t_5_26_27', 7),
 ('e_t_8_10_19', 7),
 ('e_t_8_13_21', 7),
 ('e_t_8_17_18', 7),
 ('e_t_8_19_18', 7),
 ('e_t_9_18_9', 7),
 ('e_n_11_24_34', 6),
 ('e_n_11_34_21', 6),
 ('e_n_12_4_20', 6),
 ('e_n_1_13_4', 6),
 ('e_n_1_59_45', 6),
 ('e_n_2_26_21', 6),
 ('e_n_2_7_12', 6),
 ('e_n_4_24_0', 6),
 ('e_n_6_38_34', 6),
 ('e_t_10_14_9', 6),
 ('e_t_10_15_9', 6),
 ('e_t_10_22_4', 6))

In [7]:
len(F.entityId.freqList())

28451

So tens of thousands named entity as been detected, and the most frequent ones occur only 8 times!

Now the kinds of named entities:

In [8]:
F.entityKind.freqList()

(('PER', 18718),
 ('LOC', 14819),
 ('SHP', 7199),
 ('LOCderiv', 2916),
 ('ORG', 2584),
 ('RELderiv', 130))

We guess at the legend:

* `LOC` = location
* `PER` = person
* `ORG` = organisation
* `SHP` = ship
* `REL` = relationship (??)
* *xxx*`deriv` = derived from a *xxx*

We want to know what the identifier means:

* does it connect the separate words of one entity *occurrence*?
* does it connect the multiple occurrences of a single named entity throughout the text?

We'll test. 

We collect the entity occurrences. We define an entity occurrence as the first word node in a streak of words that carry the same entity id.

A streak is a sequence of words where all words have the same entity id, but some words may have no entity id.

In [19]:
eidOccs = {}

curStreak = []
curId = None

for w in F.otype.s("word"):
    eid = F.entityId.v(w)
    
    if eid is None:
        continue
        
    if eid == curId:
        curStreak.append(w)
        continue
        
    if curId is not None:
        eidOccs.setdefault(curId, []).append(tuple(curStreak))
        
    curId = eid
    curStreak = [w]
    
if curId is not None:
    eidOccs.setdefault(curId, []).append(tuple(curStreak))
    
len(eidOccs)

28451

We only have to count the maximum number of occurrences that an entity id has, to decide what the id means.

In [20]:
max(len(occs) for occs in eidOccs.values())

3

That's interesting, let's make a count of the distribution.

In [21]:
eidFreq = collections.Counter()

for occs in eidOccs.values():
    eidFreq[len(occs)] += 1
    
eidFreq

Counter({1: 28411, 2: 37, 3: 3})

The vast majority of entities has a single occurrence.

Let's inspect the ones with multiple occurrences.

We make a table of them and where they occur.

In [29]:
for (eid, occs) in eidOccs.items():
    if len(occs) < 2:
        continue
    
    table = []
    highlights = set()
    
    ekind = F.entityKind.v(occs[0][0])
    
    for occ in occs:
        firstWord = occ[0]
        lastWord = occ[-1]
        highlights |= set(occs)
        line = L.u(firstWord, "line")[0]
        table.append((line, *tuple(range(firstWord, lastWord + 1))))
    
    A.dm(f"""\n\n### `{ekind}` entity `{eid}` ({len(occs)}x)\n\n""")
    A.table(table, highlights=highlights, extraFeatures="entityId entityKind")
    # A.show(table, highlights=highlights, extraFeatures="entityId entityKind")



### `PER` entity `e_t_1_40_10` (2x)



n,p,line,word,word
1,1 97:2,,WILLEM,JANSZ
2,1 97:26,,KASTEEL,JAKATRA




### `LOC` entity `e_n_1_68_3` (2x)





### `PER` entity `e_n_1_89_17` (2x)





### `SHP` entity `e_t_1_99_32` (2x)



...

n,p,line,word
1,13 483:7,,Amboina
2,13 483:13,,Banda
3,13 483:28,,Makassar




### `LOC` entity `e_t_13_15_22` (2x)



n,p,line,word
1,13 501:7,,Amboina
2,13 501:35,,Vermits




### `LOC` entity `e_t_13_16_24` (2x)



n,p,line,word
1,13 620:18,,Djambi
2,13 620:21,,Djambi


Let's query all words that have an entity notation:

In [7]:
query = """
word entityId entityKind*
"""
results = A.search(query)

  1.60s 32249 results


Here we query all words where the `entityId` is present.
We also mention the `entityKind` feature, but with a `*` behind it.
That is a criterion that is always True, so these mentions do not alter the result list.
But now these features do occur in the query, and hence will be shown in the results.

In [8]:
A.show(results, condensed=True, end=10)

---

# Contents

* **[start](start.ipynb)** start computing with this corpus
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[compute](compute.ipynb)** sink down a level and compute it yourself
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[annotate](annotate.ipynb)** export text, annotate with BRAT, import annotations
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **entities** use results of third-party NER (named entity recognition)
* **[porting](porting.ipynb)** port features made against an older version to a newer version
* **[volumes](volumes.ipynb)** work with selected volumes only

CC-BY Dirk Roorda